In [1]:
import data_loader as dl
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn import ensemble

In [2]:
df = dl.load_1d_grays()
df

,index,string,CC,D,Y,gray_value,filename
0,1,1836,0,3,6,"[[[166.0], [167.0], [166.0], [162.0], [164.0],...",1.jpg
1,2,1836,0,3,6,"[[[169.0], [168.0], [169.0], [170.0], [166.0],...",2.jpg
2,3,1840,0,4,0,"[[[167.0], [169.0], [169.0], [166.0], [165.0],...",3.jpg
3,4,1840,0,4,0,"[[[167.0], [165.0], [165.0], [165.0], [163.0],...",4.jpg
4,5,1823,0,2,3,"[[[62.0], [45.0], [45.0], [37.0], [40.0], [33....",5.jpg
...,...,...,...,...,...,...,...
11995,11996,1808,0,0,8,"[[[88.0], [62.0], [60.0], [68.0], [61.0], [101...",11996.jpg
11996,11997,1809,0,0,9,"[[[173.0], [171.0], [173.0], [175.0], [174.0],...",11997.jpg
11997,11998,1840,0,4,0,"[[[189.0], [189.0], [189.0], [189.0], [189.0],...",11998.jpg
11998,11999,1840,0,4,0,"[[[191.0], [192.0], [192.0], [192.0], [193.0],...",11999.jpg


In [3]:
df['gray_value'][0].shape

(56, 106, 1)

In [4]:

arr = np.zeros(shape=(12000,56,106))
i = 0
while i < 12000:
    arr[i] = np.squeeze(df['gray_value'][i])
    i += 1
arr.shape


(12000, 56, 106)

In [5]:
n_samples = len(arr)
x_values = arr.reshape((n_samples, -1))
x_values.shape

(12000, 5936)

In [43]:
from sklearn.model_selection import train_test_split
y_values = df['D']
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.2, random_state=2022)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2022)
print(x_train.shape, x_val.shape, x_test.shape)


(7680, 5936) (1920, 5936) (2400, 5936)


In [34]:

results = []
n_iterations = [100,200,500]
lr = [0.1,0.01]
n_depth = [2,4]
loss_f = 'MultiClass'

for iters in n_iterations:
    for ls in lr:
        for depths in n_depth:
            model = CatBoostClassifier(iterations=iters,
                                        learning_rate=ls,
                                        depth=depths,
                                        loss_function=loss_f)

            model.fit(x_train, y_train, verbose=50)
            acc = model.score(x_val,y_val)
            # the results from the different hyperparameters are then stored so we can get the best one
            results.append([acc,ls,iters, depths])
results_df = pd.DataFrame(results)
results_df.columns=['accuracy',"learning_rate","n_estimators", "depth"]
#prints the results from best to worst in regards to accuracy, listting the hyperparameters for the result
results_df = results_df.sort_values('accuracy', ascending=False)
results_df

0:	learn: 2.1000270	total: 345ms	remaining: 3.11s
9:	learn: 1.7137404	total: 2.6s	remaining: 0us


,accuracy,learning_rate,n_estimators,depth
0,0.261458,0.1,10,2


In [19]:
test_acc = model.score(x_test,y_test)
test_acc

0.3570833333333333

In [18]:
results_df.to_csv('Catboost_results_YEAR')

In [74]:
def score_calc(model):
    score = 0.0
    preds = model.predict(x_test)
    i = 0
    for row in y_test:
        if str(row) == str(preds[i][0]):
            score = score + (1/3)
        i= i + 1
    return score

test_score = score_calc()
test_score

234.00000000000227

In [73]:
test_score

234.00000000000227